**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(FD_RES, "source", "annotation_chipseq_tf")
for (x in dir(fdiry)){cat(x, "\n")}

ENCODE4_TF_Accessions_for_Flagship_update221025.csv 
K562.ENCFF336UPT.CTCF.bw 
K562.ENCSR000EGM.ENCFF336UPT.CTCF.bw 
K562.ENCSR000EGM.ENCFF660GHM.CTCF.bed.gz 
K562.ENCSR000EGW.ENCFF067OLL.SMC3.bw 


## Import data

In [3]:
fdiry = file.path(FD_RES, "source", "annotation_chipseq_tf")
fname = "ENCODE4_TF_Accessions_for_Flagship_update221025.csv"
fpath = file.path(fdiry, fname)

dat = read_csv(fpath)

dat_chipseq_tf_import = dat
print(dim(dat))
head(dat)

Rows: 3092 Columns: 6
── Column specification ──────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (6): Experiment_Accession, Peak_Accession, Biosample, Target, Lab, RFA

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 3092    6


Experiment_Accession,Peak_Accession,Biosample,Target,Lab,RFA
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ENCSR753RME,ENCFF917CYG,Homo sapiens testis tissue male adult (37 years),CTCF,"Bradley Bernstein, Broad",ENCODE4
ENCSR992XTY,ENCFF112GJQ,Homo sapiens WTC11,CTCF,"Bradley Bernstein, Broad",ENCODE4
ENCSR934GQS,ENCFF483KVM,Homo sapiens ovary tissue female adult (41 years),CTCF,"Bradley Bernstein, Broad",ENCODE4
ENCSR164ASX,ENCFF238PEB,Homo sapiens upper lobe of right lung tissue male adult (60 years),CTCF,"Bradley Bernstein, Broad",ENCODE4
ENCSR000VGD,ENCFF870ZRR,Homo sapiens right lobe of liver tissue female adult (47 years),CTCF,"Bradley Bernstein, Broad",ENCODE4
ENCSR856TKC,ENCFF072KJO,Homo sapiens natural killer cell male adult (33 years),CTCF,"Bradley Bernstein, Broad",ENCODE4


## Select for K562 only

In [4]:
dat = dat_chipseq_tf_import
dat = dat %>% dplyr::filter(str_detect(Biosample, "K562"))

dat_chipseq_tf_k562_full = dat
print(dim(dat))
head(dat)

[1] 735   6


Experiment_Accession,Peak_Accession,Biosample,Target,Lab,RFA
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ENCSR805SIA,ENCFF500BWO,Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens PURB,PURB,"Richard Myers, HAIB",ENCODE4
ENCSR125RFR,ENCFF863ZFH,Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens ATF6,ATF6,"Richard Myers, HAIB",ENCODE4
ENCSR841GLE,ENCFF515LWL,Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens ZNF217,ZNF217,"Richard Myers, HAIB",ENCODE4
ENCSR014ARU,ENCFF121HYT,Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens ATF2,ATF2,"Richard Myers, HAIB",ENCODE4
ENCSR172XJS,ENCFF169QYL,Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens ZNF165,ZNF165,"Michael Snyder, Stanford",ENCODE4
ENCSR676KDJ,ENCFF528IDR,Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens ARID4B,ARID4B,"Michael Snyder, Stanford",ENCODE4


In [5]:
dat = dat_chipseq_tf_k562_full
vec = unique(dat$Biosample)
print(length(vec))

[1] 278


In [6]:
string  = vec
pattern = "Homo sapiens K562 genetically modified \\(insertion\\) using CRISPR targeting"
sum(str_detect(string, pattern))

[1] 223

In [7]:
string  = vec
pattern = "Homo sapiens K562 stably expressing"
sum(str_detect(string, pattern))

[1] 54

In [12]:
sum(vec == "Homo sapiens K562")

[1] 1

In [10]:
n1 = sum(str_detect(vec, "Homo sapiens K562 genetically modified \\(insertion\\) using CRISPR targeting"))
n2 = sum(str_detect(vec, "Homo sapiens K562 stably expressing"))
n3 = sum(vec == "Homo sapiens K562")
n1 + n2 + n3

[1] 278

## TF Subset

In [11]:
targets = c(
    "POLR2A", "JUN", "CTCF", "SP1", "ELK1", "NRF1", 
    "TAL1", "GATA1", "NFE2", "KLF10", "KLF16", "YY1", 
    "ATF1", "EP300", "REST", "ZEB2"
)

idx_accession_exp = c(
    "ENCSR388QZF",
    "ENCSR000EFS",
    "ENCSR000EGM",
    "ENCSR991ELG",
    "ENCSR000EFU",
    "ENCSR998AJK",
    "ENCSR000EHB",
    "ENCSR000EFT",
    "ENCSR552YGL",
    "ENCSR121SPB",
    "ENCSR760UVO",
    "ENCSR000BMH",
    "ENCSR091GVJ",
    "ENCSR000EGE",
    "ENCSR000BMW",
    "ENCSR004GKA"
)
idx_accession_peak = c(
    "ENCFF355MNE",
    "ENCFF190CGV",
    "ENCFF660GHM",
    "ENCFF553GPK",
    "ENCFF715WGN",
    "ENCFF777PKJ",
    "ENCFF852ZRK",
    "ENCFF657CTC",
    "ENCFF023IFO",
    "ENCFF142ZTD",
    "ENCFF488OTN",
    "ENCFF398UQZ",
    "ENCFF627RSK",
    "ENCFF702XPO",
    "ENCFF707MDI",
    "ENCFF242AOL"
)

print(length(targets))
print(length(idx_accession_exp))
print(length(idx_accession_peak))

[1] 16
[1] 16
[1] 16


In [12]:
dat = dat_chipseq_tf_k562_full
dat = dat %>%
    dplyr::filter(str_detect(Biosample, "K562")) %>%
    dplyr::filter(Target %in% targets) %>%
    dplyr::filter(Peak_Accession %in% idx_accession_peak) %>%
    dplyr::mutate(Target = factor(Target, levels=targets)) %>%
    dplyr::arrange(Target)

print(all(dat$Experiment_Accession == idx_accession_exp))
print(all(dat$Peak_Accession       == idx_accession_peak))
print(all(dat$Target               == targets))

dat_chipseq_tf_k562_subset = dat
print(dim(dat))
dat

[1] TRUE
[1] TRUE
[1] TRUE
[1] 16  6


Experiment_Accession,Peak_Accession,Biosample,Target,Lab,RFA
<chr>,<chr>,<chr>,<fct>,<chr>,<chr>
ENCSR388QZF,ENCFF355MNE,Homo sapiens K562,POLR2A,"Michael Snyder, Stanford",ENCODE3
ENCSR000EFS,ENCFF190CGV,Homo sapiens K562,JUN,"Michael Snyder, Stanford",ENCODE2
ENCSR000EGM,ENCFF660GHM,Homo sapiens K562,CTCF,"Michael Snyder, Stanford",ENCODE2
ENCSR991ELG,ENCFF553GPK,Homo sapiens K562,SP1,"Michael Snyder, Stanford",ENCODE3
ENCSR000EFU,ENCFF715WGN,Homo sapiens K562,ELK1,"Michael Snyder, Stanford",ENCODE2
ENCSR998AJK,ENCFF777PKJ,Homo sapiens K562,NRF1,"Michael Snyder, Stanford",ENCODE3
ENCSR000EHB,ENCFF852ZRK,Homo sapiens K562,TAL1,"Michael Snyder, Stanford",ENCODE2
ENCSR000EFT,ENCFF657CTC,Homo sapiens K562,GATA1,"Michael Snyder, Stanford",ENCODE2
ENCSR552YGL,ENCFF023IFO,Homo sapiens K562 stably expressing NFE2,NFE2,"Michael Snyder, Stanford",ENCODE3


## Add CMD

In [13]:
lst = list(dat_chipseq_tf_k562_full, dat_chipseq_tf_k562_subset)
lst = lapply(lst, function(dat){
    dat = dat %>% 
    dplyr::mutate(
        FN_INP = paste(
            Peak_Accession,
            "bed.gz",
            sep = "."),
        FN_OUT = paste(
            "K562", 
            Experiment_Accession, 
            Peak_Accession,
            Target,
            "bed.gz",
            sep = ".")) %>%
    dplyr::mutate(
        URL = file.path(
            "https://www.encodeproject.org",
            Peak_Accession,
            "@@download",
            FN_INP)) %>%
    dplyr::mutate(
        CMD = paste(
            "wget",
            URL,
            "-O",
            FN_OUT))
})

lst_chipseq_tf_k562_cmd = lst
print(dim(lst[[1]]))
print(dim(lst[[2]]))

[1] 735  10
[1] 16 10


In [14]:
dat = lst[[2]]
head(dat)

Experiment_Accession,Peak_Accession,Biosample,Target,Lab,RFA,FN_INP,FN_OUT,URL,CMD
<chr>,<chr>,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ENCSR388QZF,ENCFF355MNE,Homo sapiens K562,POLR2A,"Michael Snyder, Stanford",ENCODE3,ENCFF355MNE.bed.gz,K562.ENCSR388QZF.ENCFF355MNE.POLR2A.bed.gz,https://www.encodeproject.org/ENCFF355MNE/@@download/ENCFF355MNE.bed.gz,wget https://www.encodeproject.org/ENCFF355MNE/@@download/ENCFF355MNE.bed.gz -O K562.ENCSR388QZF.ENCFF355MNE.POLR2A.bed.gz
ENCSR000EFS,ENCFF190CGV,Homo sapiens K562,JUN,"Michael Snyder, Stanford",ENCODE2,ENCFF190CGV.bed.gz,K562.ENCSR000EFS.ENCFF190CGV.JUN.bed.gz,https://www.encodeproject.org/ENCFF190CGV/@@download/ENCFF190CGV.bed.gz,wget https://www.encodeproject.org/ENCFF190CGV/@@download/ENCFF190CGV.bed.gz -O K562.ENCSR000EFS.ENCFF190CGV.JUN.bed.gz
ENCSR000EGM,ENCFF660GHM,Homo sapiens K562,CTCF,"Michael Snyder, Stanford",ENCODE2,ENCFF660GHM.bed.gz,K562.ENCSR000EGM.ENCFF660GHM.CTCF.bed.gz,https://www.encodeproject.org/ENCFF660GHM/@@download/ENCFF660GHM.bed.gz,wget https://www.encodeproject.org/ENCFF660GHM/@@download/ENCFF660GHM.bed.gz -O K562.ENCSR000EGM.ENCFF660GHM.CTCF.bed.gz
ENCSR991ELG,ENCFF553GPK,Homo sapiens K562,SP1,"Michael Snyder, Stanford",ENCODE3,ENCFF553GPK.bed.gz,K562.ENCSR991ELG.ENCFF553GPK.SP1.bed.gz,https://www.encodeproject.org/ENCFF553GPK/@@download/ENCFF553GPK.bed.gz,wget https://www.encodeproject.org/ENCFF553GPK/@@download/ENCFF553GPK.bed.gz -O K562.ENCSR991ELG.ENCFF553GPK.SP1.bed.gz
ENCSR000EFU,ENCFF715WGN,Homo sapiens K562,ELK1,"Michael Snyder, Stanford",ENCODE2,ENCFF715WGN.bed.gz,K562.ENCSR000EFU.ENCFF715WGN.ELK1.bed.gz,https://www.encodeproject.org/ENCFF715WGN/@@download/ENCFF715WGN.bed.gz,wget https://www.encodeproject.org/ENCFF715WGN/@@download/ENCFF715WGN.bed.gz -O K562.ENCSR000EFU.ENCFF715WGN.ELK1.bed.gz
ENCSR998AJK,ENCFF777PKJ,Homo sapiens K562,NRF1,"Michael Snyder, Stanford",ENCODE3,ENCFF777PKJ.bed.gz,K562.ENCSR998AJK.ENCFF777PKJ.NRF1.bed.gz,https://www.encodeproject.org/ENCFF777PKJ/@@download/ENCFF777PKJ.bed.gz,wget https://www.encodeproject.org/ENCFF777PKJ/@@download/ENCFF777PKJ.bed.gz -O K562.ENCSR998AJK.ENCFF777PKJ.NRF1.bed.gz


**Check NA**

In [15]:
dat = dat_chipseq_tf_k562_full
dat %>% dplyr::filter(Target == "FOS")

Experiment_Accession,Peak_Accession,Biosample,Target,Lab,RFA
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ENCSR000DKB,NA,Homo sapiens K562 stably expressing FOS,FOS,"Kevin White, UChicago",ENCODE2
ENCSR000FAI,ENCFF258PLH,Homo sapiens K562,FOS,"Sherman Weissman, Yale",ENCODE2


In [17]:
dat = dat_chipseq_tf_k562_full
print(dim(dat))
dat = dat %>% na.omit
print(dim(dat))
dat %>% dplyr::filter(Target == "FOS")

[1] 735   6
[1] 734   6


Experiment_Accession,Peak_Accession,Biosample,Target,Lab,RFA
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ENCSR000FAI,ENCFF258PLH,Homo sapiens K562,FOS,"Sherman Weissman, Yale",ENCODE2


## Save results

In [48]:
fdiry = file.path(FD_RES, "results", "region", "annotation_chipseq_tf")
fname = "accession_info_peaks.tsv"
fpath = file.path(fdiry, fname)

lst = lst_chipseq_tf_k562_cmd
dat = lst[[1]]
print(dim(dat))

write_tsv(dat, fpath)

[1] 735  10


In [49]:
fdiry = file.path(FD_RES, "results", "region", "annotation_chipseq_tf_subset")
fname = "accession_info_peaks.tsv"
fpath = file.path(fdiry, fname)

lst = lst_chipseq_tf_k562_cmd
dat = lst[[2]]
print(dim(dat))

write_tsv(dat, fpath)

[1] 16 10
